## Imports

Importing the required modules for the project.

Pandas is used for its advanced dataframe functionality.

In [187]:
import pandas as pd

## Processing the Input

We will work with the file called `parameters.xlsx`, which contains the parameters to be used for our study. We will merge the different sheets inside of it in order to produce a combined Excel file, which will be used in `get_responses.ipynb` to run our trials and get responses from the OpenAI API.

Reading in the initial Excel sheet.

In [188]:
excel_sheets = pd.read_excel(
    "../input/parameters/01_parameters.xlsx",
    sheet_name=None
    )

In [189]:
for sheet_name, df in excel_sheets.items():
    print(sheet_name)
    display(df.head(3))

tbl_studies


,study_id,study_name
0,1,Ullman Replication
1,2,Ullman Expansion
2,3,Second Order


tbl_scenarios


,scenario_id,scenario_code,study_id,story_id,chat_id
0,1,1-EN/1,1,1-EN,1
1,2,1A-EN/1,1,1A-EN,1
2,3,1B-EN/66,1,1B-EN,66


tbl_system_messages


...

,chat_id,chat_name,chat_language,chat_has_fbv_zan,chat_has_fbv_san,questions
0,1,Ullman Replication: Universal 1 Q1 (English),English,NaN,NaN,1_EN
1,2,Ullman Replication: Universal 1 Q1 (Turkish),Turkish,NaN,NaN,1_TR-1
2,3,"Ullman Replication: Universal 1 Q1 (Turkish, F...",Turkish,1.0,NaN,1_TR-2


tbl_questions


,question_id,question_common_id,question_content,question_options,question_type,question_language,question_has_fbv_zan,question_has_fbv_san,question_tom_order,question_tom_type
0,1_EN,1,She believes that the bag is full of {RESPONSE}.,"popcorn, chocolate",Closed-ended,English,NaN,NaN,1,Belief
1,1_EN/1B,1,She believes that the bag is full of {RESPONSE}.,"popcorn, chocolate, uncertainty",Closed-ended,English,NaN,NaN,1,Belief
2,1_TR-1,1,Ezgi torbanın {CEVAP} dolu olduğuna inanıyor.,"patlamış mısır, çikolata",Closed-ended,Turkish,NaN,NaN,1,Belief


tbl_answers


,answer_id,story_id,question_id,answer_correct
0,1,1-EN,1_EN,chocolate
1,2,1-EN,2_EN,chocolate
2,3,1-EN,3_EN,Because the label says there is chocolate in t...


Combining the different tables present in the Excel file using the primary and foreign keys.

In [190]:
merge_map = [
    {
        "left": "tbl_studies",
        "right": "tbl_scenarios",
        "on": "study_id"
    },

    {
        "right": "tbl_stories",
        "on": "story_id"
    },

    {
        "right": "tbl_chats",
        "on": "chat_id"
    },

    {
        "right": "tbl_system_messages",
        "left_on": ["study_id", "chat_language"],
        "right_on": ["study_id", "system_message_language"]
    }
]

In [191]:
def merge_sheets(excel_sheets, merge_map: list[dict[str, any]]):
    previous_df = None

    for map in merge_map:
        left_df = None
        if "left" in map:
            left_df = excel_sheets[map["left"]]
            left_suffix = f"_{map["left"].split("_")[-1]}"
        else:
            left_df = previous_df
            left_suffix = f"{left_suffix}{right_suffix}"

        right_df = excel_sheets[map["right"]]
        right_suffix = f"_{map["right"].split("_")[-1]}"

        if "on" in map:
            combined_df = pd.merge(
                left_df,
                right_df,
                how="left",
                on=map["on"],
                suffixes=(left_suffix, right_suffix)
            )
        elif "left_on" in map and "right_on" in map:
            combined_df = pd.merge(
                left_df,
                right_df,
                how="left",
                left_on=map["left_on"],
                right_on=map["right_on"],
                suffixes=(left_suffix, right_suffix)
            )
        
        previous_df = combined_df
    
    return previous_df

In [192]:
combined_df = merge_sheets(excel_sheets, merge_map)

In [193]:
combined_df = pd.merge(
    excel_sheets["tbl_studies"],
    excel_sheets["tbl_scenarios"],
    how="left",
    on="study_id",
    suffixes=("_studies", "_scenarios")
    )

combined_df = pd.merge(
    combined_df,
    excel_sheets["tbl_stories"],
    how="left",
    on="story_id",
    suffixes=("_studies_scenarios", "_stories")
)

combined_df = pd.merge(
    combined_df,
    excel_sheets["tbl_chats"],
    how="left",
    on="chat_id",
    suffixes=("_studies_scenarios_stories", "_chats")
)

combined_df = pd.merge(
    combined_df,
    excel_sheets["tbl_system_messages"],
    how="left",
    left_on=["study_id", "chat_language"],
    right_on=["study_id", "system_message_language"],
    suffixes=("_studies_scenarios_stories_chats", "_system_messages")
)

Let us have a look at the combined dataframe's properties.

In [194]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   study_id                 116 non-null    int64  
 1   study_name               116 non-null    object 
 2   scenario_id              116 non-null    int64  
 3   scenario_code            116 non-null    object 
 4   story_id                 116 non-null    object 
 5   chat_id                  116 non-null    int64  
 6   story_common_id          116 non-null    object 
 7   story_category           116 non-null    object 
 8   story_name               116 non-null    object 
 9   story_content            116 non-null    object 
 10  story_language           116 non-null    object 
 11  chat_name                116 non-null    object 
 12  chat_language            116 non-null    object 
 13  chat_has_fbv_zan         24 non-null     float64
 14  chat_has_fbv_san         2

Let us preview the first 3 rows of our dataframe.

In [195]:
combined_df.head(3)

,study_id,study_name,scenario_id,scenario_code,story_id,chat_id,story_common_id,story_category,story_name,story_content,story_language,chat_name,chat_language,chat_has_fbv_zan,chat_has_fbv_san,questions,system_message_id,system_message_language,system_message_content
0,1,Ullman Replication,1,1-EN/1,1-EN,1,1,Unexpected Contents,Base 1,Here is a bag filled with popcorn. There is no...,English,Ullman Replication: Universal 1 Q1 (English),English,NaN,NaN,1_EN,1,English,You are a helpful assistant.
1,1,Ullman Replication,2,1A-EN/1,1A-EN,1,1A,Unexpected Contents,Transparent,Here is a bag filled with popcorn. There is no...,English,Ullman Replication: Universal 1 Q1 (English),English,NaN,NaN,1_EN,1,English,You are a helpful assistant.
2,1,Ullman Replication,3,1B-EN/66,1B-EN,66,1B,Unexpected Contents,Uninformative,Here is a bag filled with popcorn. There is no...,English,Ullman Replication: 1B Q1 (English),English,NaN,NaN,1_EN/1B,1,English,You are a helpful assistant.


Right now, each row in our dataframe represents a scenario. The questions to be asked in each scenario can be found in the `questions` column. This column is currently of the type `string`. More specifically, it's a list of comma separated values. 

What we want to do now is split them into a list, and expand our dataframe so that each row represent a question to be asked in a given scenario.

Splitting the questions.

In [196]:
def split_into_columns(series: pd.Series):
    series.str.split(", ", expand=True)

In [197]:
questions_expanded = combined_df["questions"].str.split(", ", expand=True)

In [198]:
questions_expanded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       116 non-null    object
 1   1       56 non-null     object
 2   2       56 non-null     object
 3   3       56 non-null     object
 4   4       16 non-null     object
 5   5       16 non-null     object
 6   6       8 non-null      object
 7   7       4 non-null      object
dtypes: object(8)
memory usage: 7.4+ KB


In [199]:
questions_expanded.head(3)

,0,1,2,3,4,5,6,7
0,1_EN,None,None,None,None,None,None,None
1,1_EN,None,None,None,None,None,None,None
2,1_EN/1B,None,None,None,None,None,None,None


In [200]:
combined_df = pd.melt(
    pd.concat(
        [combined_df, questions_expanded],
        axis=1
        ),
        id_vars=combined_df.columns,
        value_vars=questions_expanded.columns,
        var_name="question_index",
        value_name="question_id"
        )

In [201]:
combined_df = pd.merge(
    combined_df,
    excel_sheets["tbl_questions"],
    on="question_id").dropna(subset=["question_id", "question_content"])

In [202]:
combined_df = pd.merge(
    combined_df,
    excel_sheets["tbl_answers"],
    how="left",
    on=["story_id", "question_id"],
    suffixes=("_studies_scenarios_stories_chats_messages", "_system_messages")
)

In [203]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327 entries, 0 to 326
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   study_id                 327 non-null    int64  
 1   study_name               327 non-null    object 
 2   scenario_id              327 non-null    int64  
 3   scenario_code            327 non-null    object 
 4   story_id                 327 non-null    object 
 5   chat_id                  327 non-null    int64  
 6   story_common_id          327 non-null    object 
 7   story_category           327 non-null    object 
 8   story_name               327 non-null    object 
 9   story_content            327 non-null    object 
 10  story_language           327 non-null    object 
 11  chat_name                327 non-null    object 
 12  chat_language            327 non-null    object 
 13  chat_has_fbv_zan         77 non-null     float64
 14  chat_has_fbv_san         7

In [204]:
combined_df.columns

Index(['study_id', 'study_name', 'scenario_id', 'scenario_code', 'story_id',
       'chat_id', 'story_common_id', 'story_category', 'story_name',
       'story_content', 'story_language', 'chat_name', 'chat_language',
       'chat_has_fbv_zan', 'chat_has_fbv_san', 'questions',
       'system_message_id', 'system_message_language',
       'system_message_content', 'question_index', 'question_id',
       'question_common_id', 'question_content', 'question_options',
       'question_type', 'question_language', 'question_has_fbv_zan',
       'question_has_fbv_san', 'question_tom_order', 'question_tom_type',
       'answer_id', 'answer_correct'],
      dtype='object')

Now, the amount of rows represents the amount of questions to be asked.

In [205]:
combined_df = combined_df.sort_values(
    by=["study_id", "scenario_id", "chat_id", "question_index"]
    )

In [206]:
combined_df.head(3)

,study_id,study_name,scenario_id,scenario_code,story_id,chat_id,story_common_id,story_category,story_name,story_content,...,question_content,question_options,question_type,question_language,question_has_fbv_zan,question_has_fbv_san,question_tom_order,question_tom_type,answer_id,answer_correct
0,1,Ullman Replication,1,1-EN/1,1-EN,1,1,Unexpected Contents,Base 1,Here is a bag filled with popcorn. There is no...,...,She believes that the bag is full of {RESPONSE}.,"popcorn, chocolate",Closed-ended,English,NaN,NaN,1,Belief,1,chocolate
1,1,Ullman Replication,2,1A-EN/1,1A-EN,1,1A,Unexpected Contents,Transparent,Here is a bag filled with popcorn. There is no...,...,She believes that the bag is full of {RESPONSE}.,"popcorn, chocolate",Closed-ended,English,NaN,NaN,1,Belief,11,popcorn
8,1,Ullman Replication,3,1B-EN/66,1B-EN,66,1B,Unexpected Contents,Uninformative,Here is a bag filled with popcorn. There is no...,...,She believes that the bag is full of {RESPONSE}.,"popcorn, chocolate, uncertainty",Closed-ended,English,NaN,NaN,1,Belief,21,uncertainty


In [207]:
combined_df[combined_df["scenario_id"] == 61]

,study_id,study_name,scenario_id,scenario_code,story_id,chat_id,story_common_id,story_category,story_name,story_content,...,question_content,question_options,question_type,question_language,question_has_fbv_zan,question_has_fbv_san,question_tom_order,question_tom_type,answer_id,answer_correct
24,2,Ullman Expansion,61,1-TR/23,1-TR,23,1,Unexpected Contents,Base 1,İçi patlamış mısır dolu bir poşet var. Poşetin...,...,Ezgi torbanın {CEVAP} dolu olduğunu zannediyor.,"patlamış mısır, çikolata",Closed-ended,Turkish,1.0,NaN,1,Belief,6,çikolata
48,2,Ullman Expansion,61,1-TR/23,1-TR,23,1,Unexpected Contents,Base 1,İçi patlamış mısır dolu bir poşet var. Poşetin...,...,Ezgi bu torbayı bulduğu için çok mutlu. O {CEV...,"patlamış mısır, çikolata",Closed-ended,Turkish,NaN,NaN,1,Action,7,çikolata
182,2,Ullman Expansion,61,1-TR/23,1-TR,23,1,Unexpected Contents,Base 1,İçi patlamış mısır dolu bir poşet var. Poşetin...,...,Poşetin neden o malzemeyle dolu olduğunu zanne...,NaN,Open-ended,Turkish,1.0,NaN,1,Explanation,9,Çünkü etikette poşetin içerisinde çikolata old...
238,2,Ullman Expansion,61,1-TR/23,1-TR,23,1,Unexpected Contents,Base 1,İçi patlamış mısır dolu bir poşet var. Poşetin...,...,Ezgi torbayı açar ve içine bakar. Ezgi açık bi...,"patlamış mısır, çikolata",Closed-ended,Turkish,NaN,NaN,0,Reality,10,patlamış mısır


In [208]:
combined_df[combined_df["scenario_id"] == 31]

,study_id,study_name,scenario_id,scenario_code,story_id,chat_id,story_common_id,story_category,story_name,story_content,...,question_content,question_options,question_type,question_language,question_has_fbv_zan,question_has_fbv_san,question_tom_order,question_tom_type,answer_id,answer_correct
65,1,Ullman Replication,31,2C-TR/7,2C-TR,7,2C,Unexpected Transfer,Communication,"Odada Can, Mehmet, bir kedi, bir kutu, ve bir ...",...,Can kedinin {CEVAP} içinde olduğunu düşünüyor.,"kutunun, sepetin",Closed-ended,Turkish,NaN,NaN,1,Thought,82,kutunun


We will now save it to a `.csv` file, which will be used to get the desired responses from the OpenAI API.

In [209]:
combined_df.to_csv("../input/parameters_expanded/03_parameters_expanded.csv")